In [3]:
from __future__ import absolute_import, division, print_function ,unicode_literals

# TensorFlow and tf. keras
import tensorflow as tf
from tensorflow import keras
import io

# Helper libraries
import pandas as pd 
from sklearn import preprocessing
from google.colab import files
from csv import reader
import random as rd
import numpy as np
import matplotlib . pyplot as plt
import math

## Exercice 1

In [4]:
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[rd.random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[rd.random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

## Exercice 2

In [5]:
rd.seed(703869)
network = initialize_network(3, 2, 2)
for layer in network:
	print(layer)

[{'weights': [0.7448335075237473, 0.6947797591566266, 0.3049317389263798, 0.6223997785765669]}, {'weights': [0.9116988058193602, 0.160705177749782, 0.4164613474335809, 0.3246818062476815]}]
[{'weights': [0.25672428496715505, 0.647152044851149, 0.7715240429695502]}, {'weights': [0.5049765871362001, 0.680458895855162, 0.16157800228536745]}]


## Exercice 3

In [6]:
def activate(weights, inputs):
  activation = 0
  for i in range(len(weights)):
    if i == len(weights):
      activation += weights[i]
    else:
      activation += weights[i] * inputs[i]
  return activation

def activate1(weights, inputs):
  activation = weights[-1]
  for i in range(len(weights)-1):
      activation = activation + (weights[i] * inputs[i])
  return activation

## Exercice 4

In [7]:
def transfer(activation):
  output = 1 / (1+math.exp(-activation))
  return output

## Exercice 5

In [8]:
def forward_propagate (network, row):
  inputs = row
  for layer in network :
    new_inputs = []
    for neuron in layer :
      neuron['output'] = transfer(activate1(neuron['weights'], inputs))
      new_inputs.append(neuron['output'])
    inputs = new_inputs
  return inputs

## Exercice 6

In [9]:
network = initialize_network(3, 2, 2)
inputs = [rd.randint(0, 1) for i in range(3)]
output = forward_propagate(network, inputs)
print(output)

[0.8265428353396238, 0.8179543091036666]


## Exercice 7

In [10]:
def transfer_derivative (output):
  return output * (1.0 - output)

## Exercice 8

In [11]:
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

## Exercice 9

In [12]:
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
	print(layer)

[{'weights': [0.6856065613603304, 0.2549410311055681, 0.432572317643073, 0.01478962312083365], 'output': 0.8002583023206206, 'delta': -0.010277073665784198}, {'weights': [0.7753134626349734, 0.005688278116613321, 0.28092250072107927, 0.3144442521250841], 'output': 0.7984071310925409, 'delta': -0.009234730067800366}]
[{'weights': [0.7593980864474268, 0.6899563149523318, 0.4027403985249194], 'output': 0.8265428353396238, 'delta': -0.11850126172600524}, {'weights': [0.9479145638290606, 0.899581912661545, 0.02573966008528139], 'output': 0.8179543091036666, 'delta': 0.027107524038216257}]


## Exercice 10

In [13]:
# weight = weight + learning_rate * error * input

def update_weight(network, row, l_rate):
  # update the weights for a network given an input raw of data
  for i in range(len(network)):
    inputs = row[:-1]
    if i != 0:
      inputs = [neuron['output'] for neuron in network[i-1]]
    for neuron in network[i]:
      for j in range(len(inputs)):
        neuron['weights'][j] = neuron['weights'][j] + l_rate * neuron['delta'] * inputs[j]
      neuron['weights'][-1] += l_rate * neuron['delta']

##Exercice 11

In [14]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
  for epoch in range(n_epoch):
    sum_error = 0
    for row in train :
      outputs = forward_propagate(network, row)
      expected = [0 for i in range (n_outputs)]
      expected[row[-1]] = 1
      sum_error += sum([(expected[i] - outputs[i])**2 for i in range(len(expected))])
      backward_propagate_error(network, expected)
      update_weight(network, row, l_rate)
  print ('>epoch =%d, lrate =%.3f, error =%.3f' % (epoch, l_rate, sum_error))

##Exercice 12

In [15]:
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]

n_inputs = 2
n_outputs = 2

network1 = initialize_network(n_inputs, 2, n_outputs)
train_network(network1, dataset, 0.5, 20, n_outputs)

for layer in network1:
	print(layer)

>epoch =19, lrate =0.500, error =0.998
[{'weights': [-0.7077729185273373, 0.9088568275423285, 0.3547120121449545], 'output': 0.16328593515022435, 'delta': -0.007030918104902349}, {'weights': [1.37454682879084, -1.9438437105960809, -0.6539930242016457], 'output': 0.9415282243053414, 'delta': 0.008168985449930855}]
[{'weights': [0.7097619606827543, -2.338982482400393, 0.7152381510172741], 'output': 0.207778324316505, 'delta': -0.03420166113355228}, {'weights': [-0.9414136246018647, 2.415043379608917, -0.6297538750947503], 'output': 0.8119756464730555, 'delta': 0.028705902931536804}]


##Exercice 13

In [16]:
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

##Exercice 14

In [17]:
for row in dataset:
	prediction = predict(network1, row)
	print( 'Expected=%d, Predicted=%d' % (row[-1], prediction))

Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=1, Predicted=1
Expected=1, Predicted=1
Expected=1, Predicted=1
Expected=1, Predicted=1
Expected=1, Predicted=1


##Exercice 15

In [ ]:
from google.colab import files
uploaded = files.upload()


TypeError: ignored

##Exercice 16

In [ ]:
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset


filename = 'seeds_dataset.csv'
dataset = load_csv(filename)
df = list()
for row in df:
	l=row[0].split('\t')

##Exercice 17

##Exercice 18

##Exercice 19

##Exercice 20

##Exercice 21